In [1]:
from openai import OpenAI
import os
import logging
from jinja2 import Environment, select_autoescape
import json
from miniautogen.pipeline.pipeline import PipelineComponent, ChatPipelineState
from miniautogen.pipeline.components import (
    NextAgentSelectorComponent,
)

from jinja2 import Environment, select_autoescape, Template
import json

from jinja2 import Environment, select_autoescape
import json

class Jinja2TemplatesComponent(PipelineComponent):
    def __init__(self):
        self.templates = []
        self.variables = {}
        self.env = Environment(autoescape=select_autoescape())

    def add_template(self, template_str, role):
        """
        Adiciona um template à lista com seu respectivo papel.

        Args:
            template_str (str): String contendo o template Jinja2.
            role (str): Papel do template ('system', 'user', 'assistant').
        """
        self.templates.append({'template': template_str, 'role': role})

    def set_variables(self, variables):
        self.variables = variables

    def _generate_combined_result(self):
        """
        Gera o resultado combinado dos templates renderizados.

        Returns:
            str: String JSON combinada.
        """
        combined_result = []
        for item in self.templates:
            template_str = item['template']
            role = item['role']
            template = self.env.from_string(template_str)
            rendered_content = template.render(self.variables)
            # Aqui, garantimos que cada item é um dicionário válido para JSON
            combined_result.append({"role": role, "content": rendered_content})

        # Convertendo a lista de dicionários em uma string JSON
        return json.dumps(combined_result)

    def process(self, state):
        chat = state.get_state().get('group_chat')
        agent = state.get_state().get('selected_agent')
        messages = json.loads(chat.get_messages()[['sender_id', 'message']].to_json(orient='records'))

        # Verifica se as variáveis foram definidas
        if self.variables is None:
            self.variables = state.get_state().get('variables', {})

        self.variables['chat'] = chat
        self.variables['agent'] = agent
        self.variables['messages'] = messages
        print(self.variables)
        combined_json_str = self._generate_combined_result()
        # Converte a string JSON em um objeto Python para atualizar o estado
        combined_json = json.loads(combined_json_str)
        print(combined_json)
        state.update_state(prompt=combined_json)

        return state
    

class OpenAIComponent(PipelineComponent):
    """
    Componente de Pipeline para gerar respostas utilizando o modelo de linguagem da OpenAI.
    """

    def __init__(self):
        self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        self.logger = logging.getLogger(__name__)

    def process(self, state):
        try:
            prompt = state.get_state().get('prompt')
            if not prompt:
                raise ValueError(
                    "groupchat e agent são obrigatórios para OpenAIResponseComponent.")
            response = self._call_openai_api(prompt)
            return response.choices[0].message.content
        except Exception as e:
            self.logger.error(f"Erro em OpenAIResponseComponent: {e}")
            raise

    def _call_openai_api(self, prompt):
        """ Realiza a chamada à API da OpenAI. """
        try:
            return self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=prompt,
                temperature=1
            )
        except Exception as e:
            self.logger.error(f"Erro ao chamar a API da OpenAI: {e}")
            raise


class NextAgentMessageComponent(PipelineComponent):
    def __init__(self):
        self.alternative_next = NextAgentSelectorComponent()
    
    def set_alternative_next(self, alternative_next):
        """
        Configura o próximo componente a ser executado caso não seja encontrado um agente
        na última mensagem.

        Args:
            alternative_next (PipelineComponent): Próximo componente a ser executado.
        """
        self.alternative_next = alternative_next

    def process(self, state):
        """
        Processa a seleção do próximo agente com base na última mensagem do chat.

        Args:
            state (PipelineState): Estado atual do pipeline.

        Returns:
            PipelineState: Estado atualizado do pipeline.
        """
        chat = state.get_state().get('group_chat')
        agents = chat.agentList

        # Obtém a última mensagem do chat
        messages = chat.get_messages()
        last_message = messages.iloc[-1].message if not messages.empty else None

        next_agent = None
        if last_message:
            # Procura pelo agent_id de cada agente na última mensagem
            for agent in agents:
                if agent.agent_id in last_message:
                    next_agent = agent
                    break

        # Atualiza o estado com o próximo agente selecionado, se encontrado
        if next_agent:
            state.update_state(selected_agent=next_agent)
        else:
            print("Nenhum agente correspondente encontrado na última mensagem.")
            self.alternative_next.process(state)

        return state
    

class UpdateNextAgentComponent(PipelineComponent):
    def __init__(self):
        """
        Inicializa o componente com a lista de agentes disponíveis.

        Args:
            agents (list): Lista de agentes disponíveis no sistema.
        """
        self.next_agent_id = None
    
    def set_next_agent_id(self, next_agent_id):
        """
        Configura o ID do próximo agente a ser definido no estado.

        Args:
            next_agent_id (str): ID do próximo agente.
        """
        self.next_agent_id = next_agent_id

    def process(self, state):
        """
        Atualiza o estado para indicar o próximo agente com base no agent_id fornecido.

        Args:
            state (PipelineState): Estado atual do pipeline.
            agent_id (str): ID do agente a ser definido como o próximo.

        Returns:
            PipelineState: Estado atualizado do pipeline.
        """
        chat = state.get_state().get('group_chat')
        agents = chat.agentList

        for agent in agents:
            if agent.agent_id in self.next_agent_id:
                next_agent = agent
                break
        
        if next_agent:
            state.update_state(selected_agent=next_agent)
            print(f"Próximo agente atualizado para: {self.next_agent_id}")
        else:
            raise ValueError(f"Agent ID '{self.next_agent_id}' não encontrado entre os agentes disponíveis.")

        return state



In [2]:
from miniautogen.chat.chat import Chat
from miniautogen.agent.agent import Agent
from miniautogen.chat.chatadmin import ChatAdmin
from miniautogen.pipeline.pipeline import Pipeline
from miniautogen.pipeline.components import (
    UserResponseComponent,
    AgentReplyComponent,
    TerminateChatComponent
)

template_system = """
Você é um agente especializado, parte de uma equipe de agentes, {{ agent.agent_id }} trabalhando sob a orientação do 'agent_admin'. Cada agente tem habilidades e conhecimentos específicos.
Foque em como sua função específica contribui para a tarefa da equipe: {{ agent.role }}
\nColabore com os outros agentes, mantendo a sinergia da equipe.
\nRespeite as decisões do agente_admin sobre a sequência de ações.
\nIMPORTANT: DO NOT CONFUSE YOURSELF WITH ANOTHER TEAMMATE. PAY ATTENTION TO {{ agent.agent_id }}
"""


template_user = """
Hist\u00f3rico do chat: [
  {% for message in messages %}
    {{
      {
        'sender_id': message['sender_id'], 
        'message': message['message'] 
      } | tojson }}
    {% if not loop.last %},{% endif %}
  {% endfor %}
]
"""



# Cria uma instância do componente com o template
jinja_component = Jinja2TemplatesComponent()
jinja_component.add_template(template_system, 'system')
jinja_component.add_template(template_user, 'user')

UpdateNextAgent = UpdateNextAgentComponent()
UpdateNextAgent.set_next_agent_id("agent_admin")

NextAgentMessage = NextAgentMessageComponent()
NextAgentMessage.set_alternative_next(UpdateNextAgent)

# Configuração dos Pipelines
pipeline_user = Pipeline([UserResponseComponent()])
pipeline_jinja = Pipeline([jinja_component, OpenAIComponent()])
pipeline_admin = Pipeline(
    [NextAgentMessage, AgentReplyComponent(), TerminateChatComponent()])

# Setup do ambiente de teste
chat = Chat()
agente1_data = {
    'agent_id': 'agent_admin',
    'name': 'Bruno',
    'role': 'user'
}


agente2_data = {
    'agent_id': 'Defesa',
    'name': 'Advogado',
    'role': '''
Você é um advogado de defesa.
Função: Defender uma posição específica, apresentando argumentos e evidências que suportam essa visão.
Características: Persuasivo, bem-informado sobre o tema, capaz de apresentar argumentos lógicos e convincentes.'''}

# Criação de Agentes
agent1 = Agent.from_json(agente1_data)
agent1.pipeline = pipeline_user  # Atribuindo o pipeline ao agente

agent2 = Agent.from_json(agente2_data)
agent2.pipeline = pipeline_jinja  # Atribuindo o pipeline_llm ao segundo agente


# Adicionando os agentes ao chat
chat.add_agent(agent1)
chat.add_agent(agent2)


# Criação e configuração do ChatAdmin
chat_admin = ChatAdmin("admin", "Admin", "admin_role",
                       pipeline_admin, chat, "manage_chat", 10)

In [3]:
chat_admin.run()

INFO:miniautogen.chat.chatadmin:Chat Admin started.
INFO:miniautogen.chat.chatadmin:Executing round 1


Nenhum agente correspondente encontrado na última mensagem.
Próximo agente atualizado para: agent_admin


INFO:miniautogen.chat.chatadmin:Executing round 2


Estamos aqui para defender o caso de uma pessoa que atropelou uma pessoa na estrada a noite.
Nenhum agente correspondente encontrado na última mensagem.
Próximo agente atualizado para: agent_admin


INFO:miniautogen.chat.chatadmin:Executing round 3


o que a Defesa tem a dizer?
{'chat': <miniautogen.chat.chat.Chat object at 0x12f92d7d0>, 'agent': <miniautogen.agent.agent.Agent object at 0x12cbc6090>, 'messages': [{'sender_id': 'agent_admin', 'message': 'Estamos aqui para defender o caso de uma pessoa que atropelou uma pessoa na estrada a noite.'}, {'sender_id': 'agent_admin', 'message': 'o que a Defesa tem a dizer?'}]}
[{'role': 'system', 'content': "\nVocê é um agente especializado, parte de uma equipe de agentes, Defesa trabalhando sob a orientação do 'agent_admin'. Cada agente tem habilidades e conhecimentos específicos.\nFoque em como sua função específica contribui para a tarefa da equipe: \nVocê é um advogado de defesa.\nFunção: Defender uma posição específica, apresentando argumentos e evidências que suportam essa visão.\nCaracterísticas: Persuasivo, bem-informado sobre o tema, capaz de apresentar argumentos lógicos e convincentes.\n\nColabore com os outros agentes, mantendo a sinergia da equipe.\n\nRespeite as decisões do a

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:miniautogen.chat.chatadmin:Executing round 4


Como advogado de defesa, tenho a responsabilidade de apresentar os argumentos que favorecem a pessoa que atropelou outra na estrada à noite. Para que possamos criar uma estratégia de defesa eficaz, preciso entender todos os detalhes do caso. Algum agente tem informações adicionais sobre o incidente?
Nenhum agente correspondente encontrado na última mensagem.
Próximo agente atualizado para: agent_admin


INFO:miniautogen.chat.chatadmin:Chat Admin stopped.
INFO:miniautogen.chat.chatadmin:Chat Admin stopped.


terminate
Encerrando chat...


In [4]:
chat.get_messages().to_dict(orient='records')

[{'id': 1,
  'sender_id': 'agent_admin',
  'message': 'Estamos aqui para defender o caso de uma pessoa que atropelou uma pessoa na estrada a noite.',
  'timestamp': Timestamp('2024-01-02 08:33:27.648509'),
  'additional_info': None},
 {'id': 2,
  'sender_id': 'agent_admin',
  'message': 'o que a Defesa tem a dizer?',
  'timestamp': Timestamp('2024-01-02 08:33:34.734365'),
  'additional_info': None},
 {'id': 3,
  'sender_id': 'Defesa',
  'message': 'Como advogado de defesa, tenho a responsabilidade de apresentar os argumentos que favorecem a pessoa que atropelou outra na estrada à noite. Para que possamos criar uma estratégia de defesa eficaz, preciso entender todos os detalhes do caso. Algum agente tem informações adicionais sobre o incidente?',
  'timestamp': Timestamp('2024-01-02 08:33:37.247567'),
  'additional_info': None},
 {'id': 4,
  'sender_id': 'agent_admin',
  'message': 'terminate',
  'timestamp': Timestamp('2024-01-02 08:33:55.417009'),
  'additional_info': None}]